# Library import

In [1]:
import pandas as pd

In [2]:
ambev = pd.read_json('dataset.json')

In [3]:
ambev

,customer_code,branch_id,sales_channel,seller_code,register_date,total_price,order_id,quantity,item_code,item_total_price,unit_price,group_code,segment_code,is_churn
0,143,0,0,190,2017-11-10T00:00:00Z,1613.53,21804,10,854,292.91,25.04,0,0,0.0
1,433,0,1,153,2011-05-16T00:00:00Z,11163.69,5486,20,246,287.19,12.33,0,5,0.0
2,486,0,0,166,2018-01-24T00:00:00Z,6432.12,22662,12,1420,184.84,12.80,0,0,0.0
3,107,0,1,156,2010-07-28T00:00:00Z,831.82,3956,18,1963,189.18,10.51,0,0,0.0
4,768,0,1,218,2010-12-17T00:00:00Z,1736.48,4730,5,1786,66.87,11.82,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204423,141,0,0,78,2017-02-15T00:00:00Z,1481.02,18922,12,2624,235.19,19.03,0,0,0.0
204424,31,0,0,181,2018-05-23T00:00:00Z,1105.38,24146,1,844,100.81,100.81,0,0,0.0
204425,192,0,0,253,2014-10-20T00:00:00Z,1059.66,13285,3,1785,51.08,15.18,0,0,0.0
204426,687,0,0,190,2015-08-18T00:00:00Z,3167.71,15167,4,1412,230.32,50.56,0,0,0.0


In [24]:
ambev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204428 entries, 0 to 204427
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   customer_code     204428 non-null  int64         
 1   branch_id         204428 non-null  int64         
 2   sales_channel     204428 non-null  int64         
 3   seller_code       204428 non-null  int64         
 4   register_date     204428 non-null  datetime64[ns]
 5   total_price       204428 non-null  float64       
 6   order_id          204428 non-null  int64         
 7   quantity          204428 non-null  int64         
 8   item_code         204428 non-null  int64         
 9   item_total_price  204428 non-null  float64       
 10  unit_price        204428 non-null  float64       
 11  group_code        204428 non-null  int64         
 12  segment_code      204428 non-null  int64         
 13  is_churn          202513 non-null  boolean       
dtypes: b

Here’s the description of each column:

 customer_code: unique id of a customer;

 branch_id: the branch id where this order was made;

 sales_channel: the sales channel this order was made;

 seller_code: seller that made this order;

 register_date: date of the order;

 total_price: total price of the order (sum of all items);

 order_id: id of this order. A order is formed by a set of items;

 item_code: code of the item;

 quantity: quantity of items, given by item_code, were bought;

 item_total_price: total price of items, i.e., quantity* price;

 unit_price: unit price of this item;

 group_code: which group this customer belongs;

 segment_code: segment this client belongs;

 is_churn: True, if we believe the client will not come back. For a given customer_code this value is always the same, it means that "today" (the day you are doing this test) this client is a churn.

# Question 1 (10 Points):

List as many use cases for the dataset as possible.

- We could make a top customers (customer_code) chart (in total_price, or count(order_id)) and understand what these customers buy (item_code) in order to market these products to others
- We could make a recommendation model (memory-based) based on client taste similarity ("collaborative filtering") to match customers with similar taste and suggest each other products they haven't bought yet.
- We could check the variation of sales (a boxplot) of each type of sales channel to see which channel is the busiest for selling (sales_channel and total_price)
- We could check the sellers (seller_code) which have the highest churn rate (is_churn) so we could verify sellers who are not good for our business. We could also check the products related to those churn.
- We could reclassify our clients according to their products based on a clustering model.
-*__We could do a ML model to predict the churn customers that are missing in our database.__*

# Question 2 (10 Points):

Pick one of the use cases you listed in question 1 and describe how building a statistical model based on the dataset could best be used to improve the business this data comes from.

- We are going to pick the case where we want to train the model and predict if a client will be churn or not, since we have missing values on that column. This could help us identifying which customers we need to focus our client satisfaction program actions. With the classification done, we can check any correlations between the churn rate and variables present in the purchase (like identifying if customers who buy few items are more likely to be churn).

# Question 3 (20 Points)

Implement the model you described in question 2, preferably in Python. The code has to retrieve the data, train and test a statistical model, and report relevant performance criteria. Ideally, we should be able to replicate your analysis from your submitted source-code, so please explicit the versions of the tools and packages you are using.

## Data wrangling

Checking for nulls and the data types.

In [4]:
ambev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204428 entries, 0 to 204427
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   customer_code     204428 non-null  int64  
 1   branch_id         204428 non-null  int64  
 2   sales_channel     204428 non-null  int64  
 3   seller_code       204428 non-null  int64  
 4   register_date     204428 non-null  object 
 5   total_price       204428 non-null  float64
 6   order_id          204428 non-null  int64  
 7   quantity          204428 non-null  int64  
 8   item_code         204428 non-null  int64  
 9   item_total_price  204428 non-null  float64
 10  unit_price        204428 non-null  float64
 11  group_code        204428 non-null  int64  
 12  segment_code      204428 non-null  int64  
 13  is_churn          202513 non-null  float64
dtypes: float64(4), int64(9), object(1)
memory usage: 21.8+ MB


- *__register_date__* is in format of object instead of *__datetime__*
- *__is_churn__* is in float64 format instead of *__boolean__* (since there are two options)
    - Yes: is churn, the client won't come back
    - No: is not churn, the client will come back)
- *__is_churn__* has some nulls

In [9]:
print(ambev['is_churn'].value_counts())
print(ambev['is_churn'].dtype)

0.0    163799
1.0     38714
Name: is_churn, dtype: int64
float64


### register_date data type correction

In [10]:
ambev['register_date'].dtype

dtype('O')

In [12]:
ambev['register_date'] = ambev['register_date'].astype('datetime64')

In [14]:
ambev

,customer_code,branch_id,sales_channel,seller_code,register_date,total_price,order_id,quantity,item_code,item_total_price,unit_price,group_code,segment_code,is_churn
0,143,0,0,190,2017-11-10,1613.53,21804,10,854,292.91,25.04,0,0,0.0
1,433,0,1,153,2011-05-16,11163.69,5486,20,246,287.19,12.33,0,5,0.0
2,486,0,0,166,2018-01-24,6432.12,22662,12,1420,184.84,12.80,0,0,0.0
3,107,0,1,156,2010-07-28,831.82,3956,18,1963,189.18,10.51,0,0,0.0
4,768,0,1,218,2010-12-17,1736.48,4730,5,1786,66.87,11.82,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204423,141,0,0,78,2017-02-15,1481.02,18922,12,2624,235.19,19.03,0,0,0.0
204424,31,0,0,181,2018-05-23,1105.38,24146,1,844,100.81,100.81,0,0,0.0
204425,192,0,0,253,2014-10-20,1059.66,13285,3,1785,51.08,15.18,0,0,0.0
204426,687,0,0,190,2015-08-18,3167.71,15167,4,1412,230.32,50.56,0,0,0.0


### is_churn data type correction

In [17]:
ambev['is_churn'] = ambev['is_churn'].astype('boolean')

In [18]:
ambev

,customer_code,branch_id,sales_channel,seller_code,register_date,total_price,order_id,quantity,item_code,item_total_price,unit_price,group_code,segment_code,is_churn
0,143,0,0,190,2017-11-10,1613.53,21804,10,854,292.91,25.04,0,0,False
1,433,0,1,153,2011-05-16,11163.69,5486,20,246,287.19,12.33,0,5,False
2,486,0,0,166,2018-01-24,6432.12,22662,12,1420,184.84,12.80,0,0,False
3,107,0,1,156,2010-07-28,831.82,3956,18,1963,189.18,10.51,0,0,False
4,768,0,1,218,2010-12-17,1736.48,4730,5,1786,66.87,11.82,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204423,141,0,0,78,2017-02-15,1481.02,18922,12,2624,235.19,19.03,0,0,False
204424,31,0,0,181,2018-05-23,1105.38,24146,1,844,100.81,100.81,0,0,False
204425,192,0,0,253,2014-10-20,1059.66,13285,3,1785,51.08,15.18,0,0,False
204426,687,0,0,190,2015-08-18,3167.71,15167,4,1412,230.32,50.56,0,0,False


### is_churn nulls correction

Checking under which conditions is_churn have null values, it seems that when clients from group_code == 3 and segment_code ==7 make purchases, they don't have is_churn.

In [19]:
ambev[ambev['is_churn'].isna()]

,customer_code,branch_id,sales_channel,seller_code,register_date,total_price,order_id,quantity,item_code,item_total_price,unit_price,group_code,segment_code,is_churn
73,834,0,0,140,2014-09-30,966.87,13192,1,2790,22.31,22.31,3,7,<NA>
78,792,0,0,101,2015-06-08,3053.39,14680,48,2630,783.36,16.32,3,7,<NA>
241,721,0,0,182,2013-02-26,5074.97,9455,6,2673,70.75,11.59,3,7,<NA>
315,529,0,0,83,2015-01-22,1664.96,13842,18,795,320.09,314.64,3,7,<NA>
377,319,0,7,177,2017-09-01,828.94,20977,3,1763,119.64,39.20,3,7,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203638,792,0,0,101,2015-02-18,3081.79,14005,18,795,279.00,279.00,3,7,<NA>
203694,529,0,1,41,2012-05-07,2351.54,7624,1,532,126.43,124.28,3,7,<NA>
204121,319,0,0,177,2017-07-10,1004.02,20307,4,1767,252.42,62.03,3,7,<NA>
204240,16,0,6,150,2015-04-15,1564.63,14384,2,1785,29.84,14.92,3,7,<NA>


We can confirmt his by checking the value counts for both columns:

In [23]:
ambev[ambev['is_churn'].isna()]['group_code'].value_counts()

3    1915
Name: group_code, dtype: int64

In [21]:
ambev[ambev['is_churn'].isna()]['segment_code'].value_counts()

7    1915
Name: segment_code, dtype: int64

Let's separate our dataset into two datasets. One which we have data for is_churn and another one that we don't have.

In [25]:
ambev_churn_non_available = ambev[ambev['is_churn'].isna()]

In [26]:
ambev_churn_available = ambev[~ambev['is_churn'].isna()]

## Logistic regression
Now that we cleaned the dataset and separated the ones we have is_churn and those we don't have, 